# Overview

This tutorial shows you how the library interacts with custom mods. To do so we create a generic mod that runs code on the vehicle and game engine side.

In this tutorial we differentiate between mod and lua module:

mod - refers to a collection of files that change the behavior of the game (see the zip file)

lua module - refers to a lua 'library' that has a global name containing a table 


## Setup

These are the usual imports to create a simple scenario. As the point is to show the mod support, nothing is happening in the scenario. The mod only prints some text to the console, so take a look at the console or the log file you can find in "~\Documents\BeamNG.research". Note that the console only flushes the last line to the log file if another console output line is generated.



In [1]:
import os
import zipfile
from pathlib import Path

from beamngpy import BeamNGpy, Scenario, Vehicle, setup_logging
from beamngpy.beamngcommon import ack

In [2]:
setup_logging()
beamng = BeamNGpy('localhost', 64256)
scenario = Scenario('smallgrid', 
                    "On how to use custom mods")

etk = Vehicle('ego_vehicle', model='etk800', licence='AI', extensions=["vehicleEngineCode"])
scenario.add_vehicle(etk, pos=(0,0,0), rot_quat=(0, 0, 1, 0))

pickup = Vehicle('some_vehicle', model='pickup', licence='AI', extensions=["vehicleEngineCode"])
scenario.add_vehicle(pickup, pos=(5,0,0), rot_quat=(0, 0, 1, 0))

scenario.make(beamng)

2021-01-05 09:35:44,103 INFO     Started BeamNGpy logging.
2021-01-05 09:35:44,104 DEBUG    Determined BeamNG.* binary to be: D:\BeamNG\game\Bin64\BeamNG.drive.x64.exe


## Creating a Simple Mod

Mods can be installed by creating a zip directory in the ".../Documents/BeamNG.reasearch/mods/" folder. They have to recreate the exact same file structure as you find in the game directory in "BeamNG.research.vX.X.X.X/lua".

What happens here is that the two lua files in the BeamNGpy directory are zipped into a mod.

In [3]:
# setting up mod
myModPath = beamng.user / 'mods' / 'genericResearchMod.zip' 
geCode = 'gameEngineCode.lua'
zipGEpath = str(Path('lua') / 'ge' / 'extensions' / 'util' / geCode)
veCode = 'vehicleEngineCode.lua'
zipVEpath = str(Path('lua') / 'vehicle' / 'extensions' / veCode)
localDir = Path(os.path.abspath('.'))
with zipfile.ZipFile(str(myModPath), 'w') as ziph:
    ziph.write(localDir / geCode, arcname=zipGEpath)
    ziph.write(localDir / veCode, arcname=zipVEpath)

2021-01-05 09:35:47,046 WARNING  d:\BeamNGpy\bng\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



## Testing the mod

To test the mod we start BeamNG.research and give the python BeamNG class the location of the gameengine mod within the "genericResearchMod.zip/lua/ge/extensions/" directory. This is necessary to register the file as its own lua module within the game.

After registration, it is available as util_gameEngineCode within the game - try typing `dump(util_gameEngineCode)` in the game's command prompt

In [4]:
bng = beamng.open(extensions=["util/gameEngineCode"])
bng.load_scenario(scenario)
bng.start_scenario()

2021-01-05 09:35:50,948 INFO     Opening BeamNPy instance...
2021-01-05 09:35:50,952 INFO     Started BeamNGpy server on localhost:64256
2021-01-05 09:35:50,953 DEBUG    Starting BeamNG process: ['D:\\BeamNG\\game\\Bin64\\BeamNG.drive.x64.exe', '-console', '-rport', '64256', '-rhost', 'localhost', '-nosteam', '-physicsfps', '4000', '-lua', "registerCoreModule('util/researchGE');registerCoreModule('util/gameEngineCode')", '-userpath', 'C:\\Users\\Pascale\\Documents\\BeamNG.drive']
2021-01-05 09:35:57,601 DEBUG    Connection established. Awaiting "hello"...
2021-01-05 09:35:57,602 INFO     Started BeamNGpy communicating on ('127.0.0.1', 50700)
2021-01-05 09:36:01,835 DEBUG    Starting vehicle server for V:ego_vehicle on: localhost:64257
2021-01-05 09:36:04,200 DEBUG    Starting vehicle server for V:some_vehicle on: localhost:64258


## How to Call Lua Module Functions

BeamNG.research and BeamNGpy communicate via sockets. 
Function parameters are send with the help of python dictionaries that are available as lua tables in the game. 
The mini mod from this tutorial follows the convention the BeamNGpy library established: Every value for the 'type' key helps identifying the appropriate handler, which for 'Foo' is expected to be 'handleFoo'.
The checkmessage function in researchGE.lua and researchVE.lua checks whether a corresponding function is locally available and, if not, calls the `onSocketMessage` hook for every extension. 

## Using Acknowledgements

In order to use the beamngcommon `ack` function decorator, the first function argument has to always be the object that also handles the socket. That is either the instance of the python BeamNG class or a vehicle object. 

## Data Transfer between BeamNGpy and BeamNG

Any return value needs to be send with the help of the socket. See here an example on how to do it on the game engine side. The code for the vehicle side is the same, it just needs to be implemented in researchVE.lua.



In [5]:
@ack('FooAck')
def callFoo(bng):
    """ Executes handleFoo defined in gameEngineCode.lua.
    """
    data = dict(type='Foo', someName = 'YourName')
    bng.send(data)

@ack('BarAck')
def callBar(veh):
    """ Executes handlebar defined in VehicleEngineCode.lua.
    Here the code is executed in the VM of the etk800.
    """
    data = dict(type='Bar', text = 'lorem ipsum...' )
    veh.send(data)

def dataExchange(bng):
    """Demonstrates how to transfer in game data to the python side.
    """
    data = dict(type='GetListOfVehicleModels')
    bng.send(data)
    response = bng.recv()
    assert response['type'] == 'ListOfVehicleModels'
    print('List of spawned vehicle models: ', response['data'])

In [6]:
callFoo(bng)

In [7]:
callBar(etk)

In [8]:
dataExchange(bng)

List of spawned vehicle models:  ['etk800', 'pickup']


### Don't see anything?

If you have trouble finding the messages in the log file or command prompt, search for 'gameEngineCode' or 'vehicleEngineCode'. These are the log tags of the respective modules.

In [9]:
bng.close()

2021-01-05 09:36:29,813 INFO     Closing BeamNGpy instance...
2021-01-05 09:36:29,813 DEBUG    Killing BeamNG process...
2021-01-05 09:36:29,897 WARNING  C:\Python38-64\lib\subprocess.py:942: ResourceWarning: subprocess 7572 is still running
  _warn("subprocess %s is still running" % self.pid,

